In [1]:
### Notebook to create subset of Audioset data. 
### Download unbalanced_train_segments.csv,  eval_segments.csv from https://research.google.com/audioset/download.html and copy in Data folder.


%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import glob

/home/deepank/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:
labels = pd.read_csv('Data/unbalanced_train_segments.csv',header=2, quotechar=r'"',skipinitialspace=True)
print(labels[:10])

        # YTID  start_seconds  end_seconds  \
0  ---1_cCGK4M            0.0         10.0   
1  ---2_BBVHAA           30.0         40.0   
2  ---B_v8ZoBY           30.0         40.0   
3  ---EDNidJUA           30.0         40.0   
4  ---N4cFAE1A           21.0         31.0   
5  ---fcVQUf3E           30.0         40.0   
6  ---g9OGAhwc           30.0         40.0   
7  ---lTs1dxhU           30.0         40.0   
8  ---mO--kRQk           30.0         40.0   
9  ---qub7uxgs           30.0         40.0   

                                     positive_labels  
0  /m/01g50p,/m/0284vy3,/m/06d_3,/m/07jdr,/m/07rwm0c  
1                                           /m/09x0r  
2                                           /m/04rlf  
3              /m/02qldy,/m/02zsn,/m/05zppz,/m/09x0r  
4                                  /m/04rlf,/m/09x0r  
5                                  /m/019jd,/m/07yv9  
6                                  /m/04rlf,/m/0c1dj  
7              /m/012f08,/m/07yv9,/m/0k4j,/t/dd00134 

In [13]:
subset_labels = pd.read_csv('Data/class_labels_reqd_latest_4.csv',names=['num','label','description']) ## CSV file includes the classes
# which are to be considered 
print(subset_labels)
l_str = '|'.join(subset_labels['label'].values)
print(l_str)
labels['new_field'] = (labels['positive_labels'].str.contains(l_str))
print(labels['positive_labels'].str.contains(l_str))
print(labels[:10])
print(len(labels))

    num       label                              description
0     4   /m/01h8n0                             Conversation
1    13  /t/dd00135                        Children shouting
2    53  /m/07pbtc8                          Walk, footsteps
3    69   /m/03qtwd                                    Crowd
4    70  /m/07qfr4h      Hubbub, speech noise, speech babble
5    71  /t/dd00013                         Children playing
6   112   /m/020bb7  Bird vocalization, bird call, bird song
7   113  /m/07pggtn                             Chirp, tweet
8   318   /m/05x_td                     Air horn, truck horn
9   326    /m/04_sv                               Motorcycle
10  327    /m/0btp2             Traffic noise, roadway noise
11  344  /t/dd00065            Light engine (high frequency)
12  348  /t/dd00066            Medium engine (mid frequency)
13  351  /t/dd00130                          Engine starting
14  352  /m/07pb8fc                                   Idling
15  500   /m/028v0c     

In [14]:
#### creation of evaluation set

labels = pd.read_csv('Data/eval_segments.csv',header=2, quotechar=r'"',skipinitialspace=True)
print(labels[:3])
labels['new_field'] = labels['positive_labels'].str.contains(l_str)
print(labels[:3])
positive = labels[labels['new_field']==True]
print(positive[:3])
positive.to_csv('Data/eval_training_subset.csv')
print(positive.shape[0])

###Download Audioset v1 embeddings from Audioset website: storage.googleapis.com/us_audioset/youtube_corpus/v1/features/features.tar.gz

files = glob.glob('audioset_v1_embeddings/eval/*')
subset_ids = positive['# YTID'].values
print(len(subset_ids))

        # YTID  start_seconds  end_seconds  \
0  --4gqARaEJE            0.0         10.0   
1  --BfvyPmVMo           20.0         30.0   
2  --U7joUcTCo            0.0         10.0   

                         positive_labels  
0  /m/068hy,/m/07q6cd_,/m/0bt9lr,/m/0jbk  
1                               /m/03l9g  
2                              /m/01b_21  
        # YTID  start_seconds  end_seconds  \
0  --4gqARaEJE            0.0         10.0   
1  --BfvyPmVMo           20.0         30.0   
2  --U7joUcTCo            0.0         10.0   

                         positive_labels  new_field  
0  /m/068hy,/m/07q6cd_,/m/0bt9lr,/m/0jbk      False  
1                               /m/03l9g      False  
2                              /m/01b_21      False  
         # YTID  start_seconds  end_seconds                positive_labels  \
5   -0CamVQdP_Y            0.0          6.0   /m/04rlf,/m/07pbtc8,/m/09x0r   
26  -1pRmoJIGQc           10.0         20.0  /m/01h82_,/m/02mk9,/m/07pb8fc   
35  -3-J

In [15]:
i=0
writer = tf.python_io.TFRecordWriter('Data/eval_subset_latest_4.tfrecord')
for tfrecord in files:
    for example in tf.python_io.tf_record_iterator(tfrecord):
        tf_example = tf.train.Example.FromString(example)
        vid_id = tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding = 'UTF-8')
        if vid_id in subset_ids:
            writer.write(example)
            i+=1
print(i)

writer.close()

1010


In [16]:
### creation of training set

labels = pd.read_csv('Data/unbalanced_train_segments.csv',header=2, quotechar=r'"',skipinitialspace=True)
print(labels[:3])
labels['new_field'] = labels['positive_labels'].str.contains(l_str)
print(labels[:3])
positive = labels[labels['new_field']==True]
print(positive[:3])
positive.to_csv('Data/train_training_subset.csv')
print(positive.shape[0])

files = glob.glob('audioset_v1_embeddings/unbal_train/*')
subset_ids = positive['# YTID'].values
print(len(subset_ids))

        # YTID  start_seconds  end_seconds  \
0  ---1_cCGK4M            0.0         10.0   
1  ---2_BBVHAA           30.0         40.0   
2  ---B_v8ZoBY           30.0         40.0   

                                     positive_labels  
0  /m/01g50p,/m/0284vy3,/m/06d_3,/m/07jdr,/m/07rwm0c  
1                                           /m/09x0r  
2                                           /m/04rlf  
        # YTID  start_seconds  end_seconds  \
0  ---1_cCGK4M            0.0         10.0   
1  ---2_BBVHAA           30.0         40.0   
2  ---B_v8ZoBY           30.0         40.0   

                                     positive_labels  new_field  
0  /m/01g50p,/m/0284vy3,/m/06d_3,/m/07jdr,/m/07rwm0c      False  
1                                           /m/09x0r      False  
2                                           /m/04rlf      False  
          # YTID  start_seconds  end_seconds  \
55   --2XRMjyizo            0.0         10.0   
62   --330hg-Ocw           30.0         40.0   
11

In [17]:
i=0
writer = tf.python_io.TFRecordWriter('Data/train_subset_latest_4.tfrecord')
for tfrecord in files:
    for example in tf.python_io.tf_record_iterator(tfrecord):
        tf_example = tf.train.Example.FromString(example)
        vid_id = tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding = 'UTF-8')
        if vid_id in subset_ids:
            writer.write(example)
            i+=1
print(i)

writer.close()

KeyboardInterrupt: 